# Dependencias

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
path = '../data/raw/Big_AHR.csv'

In [ ]:
df = pd.read_csv(path)
df.shape
df.head()
df.info()

#Comenzamos la limpieza

In [ ]:
df = df.drop(columns=["Unnamed: 0", "label"]) #Eliminar columnas no necesarias,
#Unnamed: 0 → índice duplicado → se elimina
#label → binaria y no la usaremos (crearemos ternario después) → se elimina
df.columns

In [ ]:
#Unificación del texto (title + review_text)
#Crear una sola columna de texto para el modelo
df[["title", "review_text"]].isnull().sum()
#Rellenar valores nulos
df["title"] = df["title"].fillna("")
df["review_text"] = df["review_text"].fillna("")
#Crear la columna text, Unimos título + reseña con un separador
df["text"] = (df["title"] + ". " + df["review_text"]).str.strip()
df[["title", "review_text", "text"]].head(5)
#Eliminar columnas originales de texto
df = df.drop(columns=["title", "review_text"])
df.head()

##Definir función de limpieza básica

Esta función:

Pasa a minúsculas

Elimina URLs

Elimina emails

Elimina caracteres raros

Normaliza espacios

In [ ]:
#Limpieza básica del texto (normalización)
import re
def clean_text_basic(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)      # URLs
    text = re.sub(r"\S+@\S+", "", text)             # emails
    text = re.sub(r"[^a-záéíóúñü\s]", " ", text)    # caracteres raros
    text = re.sub(r"\s+", " ", text)                # espacios extra
    return text.strip()
#Aplicar limpieza al dataset
df["text_clean"] = df["text"].apply(clean_text_basic)
df[["text", "text_clean"]].sample(5) #Comparación antes-despues

In [ ]:
#Revisión de textos vacíos
df[df["text_clean"].str.len() < 10].shape

##Creación de la etiqueta de sentimiento ternaria

In [ ]:
#nspeccionar la distribución de ratings
df["rating"].value_counts().sort_index()
#Definir función de etiquetado
def rating_to_sentiment(rating):
    if rating >= 4:
        return "positivo"
    elif rating == 3:
        return "neutro"
    else:
        return "negativo"
#Crear la nueva columna sentimiento
df["sentimiento"] = df["rating"].apply(rating_to_sentiment)
df[["rating", "sentimiento"]].sample(10)

In [ ]:
#Distribución de sentimientos
df["sentimiento"].value_counts(normalize=True)

In [ ]:
#Eliminación de textos muy cortos y duplicados
#Longitud del texto limpio
#Creamos una columna auxiliar con la longitud
df["text_len"] = df["text_clean"].str.len()
#Inspeccionar textos muy cortos
df[df["text_len"] < 20][["text_clean", "rating", "sentimiento"]].head(10)
#Eliminar textos muy cortos
df = df[df["text_len"] >= 20].copy()
#Eliminar duplicados
df = df.drop_duplicates(subset="text_clean")
df.shape

In [ ]:
#Eliminar columna auxiliar
df = df.drop(columns=["text_len"])

##Revisión final, selección de columnas y exportación

In [ ]:
df.columns

In [ ]:
#Seleccionar columnas finales
df_curado = df[
    ["text_clean", "sentimiento", "rating", "location", "hotel"]
].copy()
#Renombrar columnas (estándar ML / API)
df_curado = df_curado.rename(columns={
    "text_clean": "text",
    "sentimiento": "label"
})
df_curado.sample(5)

In [ ]:
#Distribución final de clases
df_curado["label"].value_counts(normalize=True)

# Exportacion


## Exportacion de dataset curado

In [ ]:
# Definir la ruta de exportación
path_processed = '../data/processed/sentiment_curated_v1.csv'

# Exportar el DataFrame curado a CSV
# index=False es crucial para evitar escribir una columna de índice innecesaria
df_curado.to_csv(path_processed, index=False)

print(f"Dataset curado exportado exitosamente a: {path_processed}")
print(f"Filas totales exportadas: {len(df_curado)}")